In [ ]:
import shapefile
import rasterio as rio
import numpy as np

from tools.landsat_utils import easy_raster, normalize_band as normalize
from tools.utils import create_polydict_ptsarr, xy2ll

import matplotlib.pyplot as plt
from tools.plot_tools import plot_latlon_lines
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [ ]:
import plotSettings
COLS = plotSettings.COLS
plotSettings.paper()

In [ ]:
%matplotlib tk

In [ ]:
# data directories
# general data directory
data_dir = '/'
# antarctica context image locations
Q3_dir = 'Quantarctica3'

In [ ]:

antarctica_outline = 'Glaciology/MEaSUREs Antarctic Boundaries/Coastline_Antarctica_v2.shp'
antarctica_latlines = 'Miscellaneous/Graticules/10dg_latitude.shp'
antarctica_lonlines = 'Miscellaneous/Graticules/45dg_longitude.shp'
antarctica_mosaic_file = 'SatelliteImagery/LIMA/LIMA_Mosaic.jp2'
amery_plot_mask_file = os.path.join(data_dir, 'GIS/AmeryMappingBoundingBox.shp')
antarctica_ice_shelves = 'IceShelf_Antarctica_v02.shp'

# inset 
AIS_outline,_ = create_polydict_ptsarr(os.path.join(Q3_dir, antarctica_outline))
AIS_outline = AIS_outline['Coastline']
lonshps = shapefile.Reader(os.path.join(Q3_dir, antarctica_lonlines)).shapes()
latshps = shapefile.Reader(os.path.join(Q3_dir, antarctica_latlines)).shapes()
ice_shelves, _ = create_polydict_ptsarr(os.path.join(Q3_dir, antarctica_ice_shelves))
amery_gl = ice_shelves['Amery']
amery_mask, ampts = create_polydict_ptsarr(amery_plot_mask_file)
amery_mask = amery_mask[0]

# bounding box locations for context image
bb_filename = os.path.join(data_dir, 'GIS/BoundingBoxes2.shp')
print('loading mask')
bbdict, bbpts = create_polydict_ptsarr(bb_filename)
mask = bbdict[0]


# Locations of specific images for figure
collapse_dirs = ['data/LandSat8-9_Color/LC08_L1GT_125111_20150329_20200909_02_T2', 
                 'data/LandSat8-9_Color/LC08_L1GT_128111_20150403_20200909_02_T2']

# files = [first known observation, last known melt, ..., ..., latest image
img_hist_dirs = ['data/LandSat1-5/LM01_L1GS_135111_19730316_20200909_02_T2',
                 'data/LandSat7/LE07_L1GT_126111_20060114_20200914_02_T2',
                 'data/LandSat8-9_Color/LC08_L1GT_128111_20190124_20201016_02_T2',
                 'data/LandSat8-9_Color/LC09_L1GT_126111_20220126_20230430_02_T2'
                ]

# bands for each landsat mision
ls1_bands = ['B4', 'B5', 'B6',]
ls4_bands  = ['B1', 'B2', 'B3']
ls7_bands = ['B1', 'B2', 'B3', 'B5', 'B8']
ls89_bands = ['B2', 'B3', 'B4', 'B6',  'B8']

In [ ]:
SAVE = False
# name to save figure as
save_name = 'figure1'
# directory to save figure
save_dir = 'figs'

#############################################################################################################
# settings
#############################################################################################################
plt.close('all')

ice_shelf_color = (137/255, 207/255, 240/255, 1) # inset plot ice shelf coloring

grounding_line_opts = {'color': 'k', 'lw': 0.5, 'ls':'-'}

 # set up figure
figsize = [16.832,  9.184]
fig = plt.figure(num=1, clear=1, figsize=figsize

NROWS = 2
NCOLS = 8

PARENT = gridspec.GridSpec(NROWS, NCOLS, figure=fig,
                           hspace=0.03,
                           left=0.03, right=0.97, bottom=0.03, top=0.97
                          )

# amery context image
gss = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=PARENT[0,:3])
amerymap = fig.add_subplot(gss[:])

# just before and just after drainage event images
gs0 = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=PARENT[0, 3:], 
                                      wspace=0.03)
pre, post = [fig.add_subplot(gs0[0]), fig.add_subplot(gs0[1])]

# observations through time                 
gs1 = gridspec.GridSpecFromSubplotSpec(1, NCOLS//2, subplot_spec=PARENT[1,:],
                                       wspace=0.03)
axs = [fig.add_subplot(gs1[i]) for i in range(NCOLS//2)]

#############################################################################################################
# amery ice shelf context image
#############################################################################################################
antarctica_mosaic = rio.open(os.path.join(Q3_dir, antarctica_mosaic_file))
amery, _ = rio.mask.mask(antarctica_mosaic,
                         [amery_mask],
                         crop=True,
                         filled=False)

left, bottom, right, top = amery_mask.bounds
lon, lat = xy2ll(ampts[:, 0], ampts[: ,1])
lon_range = [np.min(lon), np.max(lon)]
lat_range = [np.min(lat), np.max(lat)]
RGB = np.zeros((amery.shape[1], amery.shape[2], amery.shape[0]))

RGB[:,:,0] = amery[0]/amery[0].max()
RGB[:,:,1] = amery[1]/amery[1].max()
RGB[:,:,2] = amery[2]/amery[2].max()
MIN = [0,0,0]
MAX = [np.quantile(a, 0.98) for a in [amery[0], amery[1], amery[2]]]


amerymap.imshow(RGB, 
                extent=(left, right, bottom, top),
                rasterized=True,
               )#aspect='auto')

# plot grounding lines
glx = amery_gl.exterior.xy[0]
gly = amery_gl.exterior.xy[1]

amerymap.plot(glx, gly,
              **grounding_line_opts)
for intr in amery_gl.interiors:
    amerymap.plot(intr.xy[0], intr.xy[1],
                  **grounding_line_opts)
    
# plot doline bounding box
amerymap.plot(mask.exterior.xy[0], mask.exterior.xy[1],
           lw=2, color=COLS[1])

amerymap.set(ylim=(bottom, top), xlim=(left, right),
          yticklabels=[], xticklabels=[],
          xticks=[], yticks=[])
# plot lat/lon lines
amerymap = plot_latlon_lines(amerymap,
                          extents=(left, right, bottom, top),
                          lon_range=lon_range, lat_range=lat_range,
                          lon_step=2, lat_step=1,
                          ignore_lon_intersect=False,
                          )


# add a scale bar
scalebar = AnchoredSizeBar(amerymap.transData,
                           50e3, '50 km', 'lower left', 
                           pad=1,
                           #color='white',
                           frameon=False,
                           size_vertical=1e3,
                          )#fontproperties=fontprops)

amerymap.add_artist(scalebar)
[a.set_linewidth(1.2) for _,a in amerymap.spines.items()]

# antarctica outline inset axis
aoax = inset_axes(amerymap, width='20%', height='30%', loc='upper right')


aoax.fill(AIS_outline.boundary.xy[0], AIS_outline.boundary.xy[1],
          fc='grey', 
          alpha=0.25, ec='k',lw=0.1)
for shfl, poly in ice_shelves.items():
    aoax.fill(poly.exterior.xy[0], poly.exterior.xy[1], 
              fc=ice_shelf_color, alpha=1, ec='k', lw=0.1)

# plot the grounding line
aoax.plot(amery_mask.boundary.xy[0], amery_mask.boundary.xy[1], c=COLS[0], lw=1.5)

aoax.set(yticklabels=[], xticklabels=[],
         yticks=[], xticks=[],
         ylim=aoax.get_ylim(), xlim=aoax.get_xlim()
        )
[i.set_linewidth(1) for _,i in aoax.spines.items()]

# plotting lat lon
for v in lonshps:
    pts = np.array(v.points)
    aoax.plot(pts[:,0], pts[:,1], color='grey', lw=0.3, ls='--')
for v in latshps:
    pts = np.array(v.points)
    aoax.plot(pts[:,0], pts[:,1], color='grey', lw=0.3, ls='--')
    
#############################################################################################################
# collapse imagery
#############################################################################################################

bands = ls89_bands
red,green,blue,pan,SWIR = ls89_rgbps
pan_sharpen_method='brovey'
for i, ax in enumerate([pre, post]):
    rast = easy_raster(os.path.join(data_dir, collapse_dirs[i]),
                       bands=bands,
                       mask=mask,
                       always_TOA=True)
    ps = rast.pan_sharpen(method=pan_sharpen_method,
                          red=red, green=green, blue=blue, pan=pan, SWIR=SWIR, 
                          TOA_correction=True)
    left, bottom, right, top = mask.bounds
    R,G,B = rast.masked[red],  rast.masked[green], rast.masked[blue]
    MIN = [0,0,0]
    MAX = [np.quantile(rast.bands[b], 0.98) for b in [red, green, blue]]
    shape = R.shape
    # contrast stretch RGB image
    RGB = np.empty((shape[0], shape[1], 3),dtype=int)
    RGB[:,:,0] = np.clip(255*normalize(R, MIN[0], MAX[0]), 0, 255)
    RGB[:,:,1] = np.clip(255*normalize(G, MIN[1], MAX[1]), 0, 255)
    RGB[:,:,2] = np.clip(255*normalize(B, MIN[2], MAX[2]), 0, 255)
    ax.imshow(RGB, extent=(left, right, bottom, top), rasterized=True)
    ax = plot_latlon_lines(ax, lat_range=lat_range, lon_range=lon_range,
                            extents=(left, right, bottom, top),
                            lon_text=True if i == 0 else False)
    ax.set(ylim=(bottom, top), xlim=(left, right),
           yticklabels=[], xticklabels=[],
           xticks=[], yticks=[])
    
    scalebar = AnchoredSizeBar(ax.transData,
                               1e3, '1 km', 'lower right', 
                               pad=1,
                               #color='white',
                               frameon=False,
                               size_vertical=100,
                              )#fontproperties=fontprops)
    [a.set_linewidth(1) for _,a in ax.spines.items()]
    ax.add_artist(scalebar)
    
                 # plot grounding line
    for intr in amery_gl.interiors:
        ax.plot(intr.xy[0], intr.xy[1],
                  **grounding_line_opts)
    
#############################################################################################################
# observations through time
#############################################################################################################
letters=['b','c','d','e','f'] # subplot letters

for i,fdir in enumerate(img_hist_dirs):
    if 'LandSat1-5' in fdir:
        bands = ls1_bands
        red,green,blue,pan,SWIR = ls1_rgbps
    elif 'LandSat4' in fdir:
        bands = ls4_bands
        red,green,blue,pan,SWIR = ls4_rgbps
    elif 'LandSat7' in fdir:
        bands = ls7_bands
        red,green,blue,pan,SWIR = ls7_rgbps
        pan_sharpen_method = 'IHS'
    elif 'LandSat8-9' in fdir:
        bands = ls89_bands
        red,green,blue,pan,SWIR = ls89_rgbps
        pan_sharpen_method='brovey'
    rast = easy_raster(os.path.join(data_dir, fdir),
                       bands=bands,
                       mask=mask,
                       always_TOA=True)
    left, bottom, right, top = mask.bounds
    if pan is not None:
        ps = rast.pan_sharpen(method=pan_sharpen_method,
                              red=red, green=green, blue=blue, pan=pan, SWIR=SWIR, 
                              TOA_correction=True)
        R, G, B = ps[red], ps[green], ps[blue]
        MIN = (0,0,0)
        MAX = [np.quantile(b, 0.98) for b in [R,G,B]]
    else:
        R,G,B = rast.masked[red],  rast.masked[green], rast.masked[blue]
        MIN = [0,0,0]
        MAX = [np.quantile(rast.bands[b], 0.98) for b in [red, green, blue]]
    shape = R.shape
    RGB = np.empty((shape[0], shape[1], 3),dtype=int)
    RGB[:,:,0] = np.clip(255*normalize(R, MIN[0], MAX[0]), 0, 255)
    RGB[:,:,1] = np.clip(255*normalize(G, MIN[1], MAX[1]), 0, 255)
    RGB[:,:,2] = np.clip(255*normalize(B, MIN[2], MAX[2]), 0, 255)
    year = rast.date.year
    month = '{0}'.format(rast.date.month).zfill(2)
    day = '{0}'.format(rast.date.day).zfill(2)
    axs[i].text(left + 100, top - 600, 
                r'$\bf{{({A})}}$'.format(A=letters[i]) + ' ' + '{0}-{1}-{2}'.format(year, month, day),
                fontsize=22, va='center', ha='left',
                transform=axs[i].transData, zorder=99)
    axs[i].imshow(RGB, extent=(left, right, bottom, top), rasterized=True)
    axs[i] = plot_latlon_lines(axs[i], lat_range=lat_range, lon_range=lon_range,
                               extents=(left, right, bottom, top),
                               lon_text=True if i == 0 else False)
    axs[i].set(ylim=(bottom, top), xlim=(left, right),
               yticklabels=[], xticklabels=[],
               xticks=[], yticks=[])
    
    scalebar = AnchoredSizeBar(axs[i].transData,
                               1e3, '1 km', 'lower right', 
                               pad=1,
                               #color='white',
                               frameon=False,
                               size_vertical=100,
                              )#fontproperties=fontprops)
    [a.set_linewidth(1) for _,a in axs[i].spines.items()]
    axs[i].add_artist(scalebar)
    # plot grounding line ? 
    for intr in amery_gl.interiors:
        axs[i].plot(intr.xy[0], intr.xy[1],
                  **grounding_line_opts)

if SAVE:
    plt.savefig(os.path.join(save_dir, save_name), dpi=600)
plt.show()